<a href="https://colab.research.google.com/github/piyush5566/Twitter-Sentiment-Analysis/blob/master/ML_CLASS_PRO_lstm_cnn_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow-text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 107.8 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
#from keras.preprocessing.text import Tokenizer #This line caused the error.
from tensorflow.keras.preprocessing.text import Tokenizer #This line fixes the error by importing Tokenizer from tensorflow.keras.preprocessing.text
from tensorflow.keras.preprocessing.sequence import pad_sequences #Updated import path for pad_sequences
from tensorflow.keras.utils import to_categorical #Updated import path for to_categorical
from tensorflow.keras.models import Sequential #Updated import path for Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, LeakyReLU, Conv1D, GlobalMaxPooling1D #Updated import paths for layers
from tensorflow.keras import optimizers #Updated import path for optimizers
from tensorflow.keras.models import load_model #Updated import path for load_model
import json, argparse, os
import re
import io
import sys

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import json, argparse, os
import re
import io
import sys
from keras.layers import Conv1D, GlobalMaxPooling1D
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
# !wget https://nlp.stanford.edu/data/glove.6B.zip

--2024-12-30 09:48:35--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-12-30 09:48:35--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.04MB/s    in 2m 39s  

2024-12-30 09:51:15 (5.16 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [7]:
# !git clone https://github.com/DhruvDh/emocontext.git

Cloning into 'emocontext'...
remote: Enumerating objects: 176, done.
remote: Total 176 (delta 0), reused 0 (delta 0), pack-reused 176 (from 1)
Receiving objects: 100% (176/176), 44.31 MiB | 22.84 MiB/s, done.
Resolving deltas: 100% (79/79), done.
Updating files: 100% (44/44), done.


In [ ]:
#!ls
#!pwd

In [9]:
trainDataPath = "/content/emocontext/data/train.txt"
testDataPath = "/content/emocontext/data/dev.txt"

# Path to directory where GloVe file is saved.
gloveDir = "/content/drive/My Drive/glove/"

NUM_FOLDS = 5                      # Value of K in K-fold Cross Validation
NUM_CLASSES = 4                    # Number of classes - Happy, Sad, Angry, Others
MAX_NB_WORDS = 20000                # To set the upper limit on the number of tokens extracted using keras.preprocessing.text.Tokenizer
MAX_SEQUENCE_LENGTH = 100           # All sentences having lesser number of words than this will be padded
EMBEDDING_DIM = 100                # The dimension of the word embeddings
BATCH_SIZE = 200                  # The batch size to be chosen for training the model.
LSTM_DIM = 128                    # The dimension of the representations learnt by the LSTM model
DROPOUT = 0.2                        # Fraction of the units to drop for the linear transformation of the inputs. Ref - https://keras.io/layers/recurrent/
NUM_EPOCHS = 20                   # Number of epochs to train a model for

LEARNING_RATE = 0.003
label2emotion = {0:"others", 1:"happy", 2: "sad", 3:"angry"}
emotion2label = {"others":0, "happy":1, "sad":2, "angry":3}


In [10]:
train_data = pd.read_csv(trainDataPath,sep = '\t')
train_data.head()

,id,turn1,turn2,turn3,label
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,2,By,by Google Chrome,Where you live,others
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others


from matplotlib import pyplot as plt
_df_0['id'].plot(kind='hist', bins=20, title='id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('turn1').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('turn2').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('turn3').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['turn2'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='id', y='turn2', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['turn3'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='id', y='turn3', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_16['label'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_16, x='id', y='label', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [13]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30160 entries, 0 to 30159
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      30160 non-null  int64 
 1   turn1   30157 non-null  object
 2   turn2   30160 non-null  object
 3   turn3   30160 non-null  object
 4   label   30160 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.2+ MB


In [14]:
train_data['label'].value_counts()

,count
label,
others,14948
angry,5506
sad,5463
happy,4243


In [ ]:
test_data = pd.read_csv(testDataPath,sep = '\t')
test_data.head()

,id,turn1,turn2,turn3,label
0,0,Then dont ask me,YOURE A GUY NOT AS IF YOU WOULD UNDERSTAND,IM NOT A GUY FUCK OFF,angry
1,1,Mixed things such as??,the things you do.,Have you seen minions??,others
2,2,Today I'm very happy,and I'm happy for you ❤,I will be marry,happy
3,3,Woah bring me some,left it there oops,Brb,others
4,4,it is thooooo,I said soon master.,he is pressuring me,others


In [ ]:
test_data['label'].value_counts()

others    2338
angry      150
happy      142
sad        125
Name: label, dtype: int64

In [ ]:
def preprocessData(dataFilePath, mode):
    """Load data from a file, process and return indices, conversations and labels in separate lists
    Input:
        dataFilePath : Path to train/test file to be processed
        mode : "train" mode returns labels. "test" mode doesn't return labels.
    Output:
        indices : Unique conversation ID list
        conversations : List of 3 turn conversations, processed and each turn separated by the <eos> tag
        labels :  List of labels
    """
    indices = []
    conversations = []
    labels = []
    with io.open(dataFilePath, encoding="utf8") as finput:
        finput.readline()
        for line in finput:
            # Convert multiple instances of . ? ! , to single instance
            # okay...sure -> okay . sure
            # okay???sure -> okay ? sure
            # Add whitespace around such punctuation
            # okay!sure -> okay ! sure
            repeatedChars = ['.', '?', '!', ',']
            for c in repeatedChars:
                lineSplit = line.split(c)
                while True:
                    try:
                        lineSplit.remove('')
                    except:
                        break
                cSpace = ' ' + c + ' '
                line = cSpace.join(lineSplit)

            line = line.strip().split('\t')

            label = emotion2label[line[4]]
            labels.append(label)

            conv = ' <eos> '.join(line[1:4])

            # Remove any duplicate spaces
            duplicateSpacePattern = re.compile(r'\ +')
            conv = re.sub(duplicateSpacePattern, ' ', conv)

            indices.append(int(line[0]))
            conversations.append(conv.lower())


    return indices, conversations, labels



In [ ]:
def getMetrics(predictions, ground):
    """Given predicted labels and the respective ground truth labels, display some metrics
    Input: shape [# of samples, NUM_CLASSES]
        predictions : Model output. Every row has 4 decimal values, with the highest belonging to the predicted class
        ground : Ground truth labels, converted to one-hot encodings. A sample belonging to Happy class will be [0, 1, 0, 0]
    Output:
        accuracy : Average accuracy
        microPrecision : Precision calculated on a micro level. Ref - https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin/16001
        microRecall : Recall calculated on a micro level
        microF1 : Harmonic mean of microPrecision and microRecall. Higher value implies better classification
    """
    # [0.1, 0.3 , 0.2, 0.1] -> [0, 1, 0, 0]
    discretePredictions = to_categorical(predictions.argmax(axis=1))

    truePositives = np.sum(discretePredictions*ground, axis=0)
    falsePositives = np.sum(np.clip(discretePredictions - ground, 0, 1), axis=0)
    falseNegatives = np.sum(np.clip(ground-discretePredictions, 0, 1), axis=0)

    print("True Positives per class : ", truePositives)
    print("False Positives per class : ", falsePositives)
    print("False Negatives per class : ", falseNegatives)

    # ------------- Macro level calculation ---------------
    macroPrecision = 0
    macroRecall = 0
    # We ignore the "Others" class during the calculation of Precision, Recall and F1
    for c in range(1, NUM_CLASSES):
        precision = truePositives[c] / (truePositives[c] + falsePositives[c])
        macroPrecision += precision
        recall = truePositives[c] / (truePositives[c] + falseNegatives[c])
        macroRecall += recall
        f1 = ( 2 * recall * precision ) / (precision + recall) if (precision+recall) > 0 else 0
        print("Class %s : Precision : %.3f, Recall : %.3f, F1 : %.3f" % (label2emotion[c], precision, recall, f1))

    macroPrecision /= 3
    macroRecall /= 3
    macroF1 = (2 * macroRecall * macroPrecision ) / (macroPrecision + macroRecall) if (macroPrecision+macroRecall) > 0 else 0
    print("Ignoring the Others class, Macro Precision : %.4f, Macro Recall : %.4f, Macro F1 : %.4f" % (macroPrecision, macroRecall, macroF1))

    # ------------- Micro level calculation ---------------
    truePositives = truePositives[1:].sum()
    falsePositives = falsePositives[1:].sum()
    falseNegatives = falseNegatives[1:].sum()

    print("Ignoring the Others class, Micro TP : %d, FP : %d, FN : %d" % (truePositives, falsePositives, falseNegatives))

    microPrecision = truePositives / (truePositives + falsePositives)
    microRecall = truePositives / (truePositives + falseNegatives)

    microF1 = ( 2 * microRecall * microPrecision ) / (microPrecision + microRecall) if (microPrecision+microRecall) > 0 else 0
    # -----------------------------------------------------

    predictions = predictions.argmax(axis=1)
    ground = ground.argmax(axis=1)
    accuracy = np.mean(predictions==ground)

    print("Accuracy : %.4f, Micro Precision : %.4f, Micro Recall : %.4f, Micro F1 : %.4f" % (accuracy, microPrecision, microRecall, microF1))
    return accuracy, microPrecision, microRecall, microF1

In [ ]:
def getEmbeddingMatrix(wordIndex):
    """Populate an embedding matrix using a word-index. If the word "happy" has an index 19,
       the 19th row in the embedding matrix should contain the embedding vector for the word "happy".
    Input:
        wordIndex : A dictionary of (word : index) pairs, extracted using a tokeniser
    Output:
        embeddingMatrix : A matrix where every row has 100 dimensional GloVe embedding
    """
    embeddingsIndex = {}
    # Load the embedding vectors from ther GloVe file
    with io.open(os.path.join(gloveDir, 'glove.6B.100d.txt'), encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            embeddingVector = np.asarray(values[1:], dtype='float32')
            embeddingsIndex[word] = embeddingVector

    print('Found %s word vectors.' % len(embeddingsIndex))

    # Minimum word index of any word is 1.
    embeddingMatrix = np.zeros((len(wordIndex) + 1, EMBEDDING_DIM))
    for word, i in wordIndex.items():
        embeddingVector = embeddingsIndex.get(word)
        if embeddingVector is not None:
            # words not found in embedding index will be all-zeros.
            embeddingMatrix[i] = embeddingVector

    return embeddingMatrix

In [ ]:
def buildModel(embeddingMatrix):
    """Constructs the architecture of the model
    Input:
        embeddingMatrix : The embedding matrix to be loaded in the embedding layer.
    Output:
        model : A basic LSTM model
    """
    embeddingLayer = Embedding(embeddingMatrix.shape[0],
                                EMBEDDING_DIM,
                                weights=[embeddingMatrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)
    model_lstm = Sequential()
    model_lstm.add(embeddingLayer)
    model_lstm.add(LSTM(LSTM_DIM, dropout=DROPOUT))
    model_lstm.add(LeakyReLU())
    model_lstm.add(Dense(64, activation = 'relu'))
    model_lstm.add(Dense(NUM_CLASSES, activation='softmax'))

    rmsprop = optimizers.rmsprop(lr=LEARNING_RATE)
    model_lstm.compile(loss='categorical_crossentropy',
                  optimizer=rmsprop,
                  metrics=['acc'])

    model_cnn = Sequential()
    model_cnn.add(embeddingLayer)
    model_cnn.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model_cnn.add(GlobalMaxPooling1D())
    model_cnn.add(Dense(256, activation='relu'))
    model_cnn.add(Dense(NUM_CLASSES, activation='softmax'))
    model_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])



    return model_lstm, model_cnn


In [ ]:








print("Processing training data...")
trainIndices, trainTexts, labels = preprocessData(trainDataPath, mode="train")

print("Processing test data...")
testIndices, testTexts, y_test = preprocessData(testDataPath, mode="test")


print("Extracting tokens...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(trainTexts)
trainSequences = tokenizer.texts_to_sequences(trainTexts)
testSequences = tokenizer.texts_to_sequences(testTexts)

wordIndex = tokenizer.word_index
print("Found %s unique tokens." % len(wordIndex))

print("Populating embedding matrix...")
embeddingMatrix = getEmbeddingMatrix(wordIndex)

data = pad_sequences(trainSequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels))
print("Shape of training data tensor: ", data.shape)
print("Shape of label tensor: ", labels.shape)

# Randomize data
np.random.shuffle(trainIndices)
data = data[trainIndices]
labels = labels[trainIndices]

X_train, X_dev, y_train, y_dev = train_test_split(
data, labels, test_size=0.10, random_state=42)








Processing training data...
Processing test data...
Extracting tokens...
Found 16831 unique tokens.
Populating embedding matrix...
Found 400000 word vectors.
Shape of training data tensor:  (30160, 100)
Shape of label tensor:  (30160, 4)


In [ ]:
import tensorflow as tf

In [ ]:



print("Building model...")
model_lstm,model_cnn = buildModel(embeddingMatrix)

callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 4, restore_best_weights = True)

print('Training LSTM model\n')
model_lstm.fit(X_train, y_train, validation_data=(X_dev, y_dev),
               epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks = [callback])

print('Training CNN model\n')

model_cnn.fit(X_train, y_train, validation_data=(X_dev, y_dev),
               epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks = [callback])




Building model...
Training LSTM model

Train on 27144 samples, validate on 3016 samples
Epoch 1/20
27144/27144 [==============================] - 22s 824us/step - loss: 0.8625 - acc: 0.6558 - val_loss: 0.6195 - val_acc: 0.7520
Epoch 2/20
27144/27144 [==============================] - 22s 809us/step - loss: 0.6220 - acc: 0.7619 - val_loss: 0.5563 - val_acc: 0.7822
Epoch 3/20
27144/27144 [==============================] - 22s 810us/step - loss: 0.5369 - acc: 0.7955 - val_loss: 0.4684 - val_acc: 0.8193
Epoch 4/20
27144/27144 [==============================] - 22s 812us/step - loss: 0.4721 - acc: 0.8221 - val_loss: 0.4788 - val_acc: 0.8163
Epoch 5/20
27144/27144 [==============================] - 22s 815us/step - loss: 0.4284 - acc: 0.8390 - val_loss: 0.4281 - val_acc: 0.8392
Epoch 6/20
27144/27144 [==============================] - 22s 822us/step - loss: 0.3940 - acc: 0.8503 - val_loss: 0.4372 - val_acc: 0.8269
Epoch 7/20
27144/27144 [==============================] - 22s 808us/step - los

In [ ]:
testData = pad_sequences(testSequences, maxlen=MAX_SEQUENCE_LENGTH)

predictions_lstm = model_lstm.predict(testData, batch_size=BATCH_SIZE)
predictions_lstm = predictions_lstm.argmax(axis=1)

print('FOR GLOVE + LSTM')
print(classification_report(y_test,predictions_lstm))

FOR GLOVE + LSTM
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      2338
           1       0.57      0.65      0.61       142
           2       0.49      0.74      0.59       125
           3       0.59      0.77      0.67       150

    accuracy                           0.88      2755
   macro avg       0.65      0.77      0.70      2755
weighted avg       0.90      0.88      0.88      2755



In [ ]:
predictions_cnn = model_cnn.predict(testData, batch_size=BATCH_SIZE)
predictions_cnn = predictions_cnn.argmax(axis=1)

print('FOR GLOVE + CNN')
print(classification_report(y_test,predictions_cnn))

FOR GLOVE + CNN
              precision    recall  f1-score   support

           0       0.95      0.87      0.91      2338
           1       0.44      0.53      0.48       142
           2       0.41      0.72      0.52       125
           3       0.54      0.82      0.65       150

    accuracy                           0.84      2755
   macro avg       0.59      0.73      0.64      2755
weighted avg       0.88      0.84      0.86      2755

